In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  # MNIST har bara grayscale
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # 10 klasser i MNIST

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

mnist_model = MNIST_CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mnist_model.parameters(), lr=0.001)


epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = mnist_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    print(f"MNIST Accuracy: {100 * correct / total:.2f}%")

test_model(mnist_model, testloader)

torch.save(mnist_model.state_dict(), "mnist_cnn.pth")


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:38<00:00, 257kB/s] 


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 256kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:04<00:00, 354kB/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Epoch 1, Loss: 0.2254
Epoch 2, Loss: 0.0583
Epoch 3, Loss: 0.0401
Epoch 4, Loss: 0.0305
Epoch 5, Loss: 0.0237
Epoch 6, Loss: 0.0204
Epoch 7, Loss: 0.0155
Epoch 8, Loss: 0.0141
Epoch 9, Loss: 0.0116
Epoch 10, Loss: 0.0102
MNIST Accuracy: 99.05%


In [2]:
svhn_transform = transforms.Compose([
    transforms.Resize((28, 28)),  
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

svhn_trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=svhn_transform)
svhn_testset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=svhn_transform)

svhn_trainloader = torch.utils.data.DataLoader(svhn_trainset, batch_size=64, shuffle=True)
svhn_testloader = torch.utils.data.DataLoader(svhn_testset, batch_size=64, shuffle=False)

svhn_model = MNIST_CNN()
svhn_model.load_state_dict(torch.load("mnist_cnn.pth"))

svhn_model.conv1 = nn.Conv2d(3, 16, 3, padding=1)

for param in svhn_model.conv1.parameters():
    param.requires_grad = False
for param in svhn_model.conv2.parameters():
    param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(svhn_model.fc1.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in svhn_trainloader:
        optimizer.zero_grad()
        outputs = svhn_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(svhn_trainloader):.4f}")

test_model(svhn_model, svhn_testloader)


100%|██████████| 182M/182M [07:44<00:00, 392kB/s]  


100%|██████████| 64.3M/64.3M [02:42<00:00, 397kB/s]
C:\Users\lindh\AppData\Local\Temp\ipykernel_10612\2357167377.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  svhn_mo

Epoch 1, Loss: 1.8372
Epoch 2, Loss: 1.4670
Epoch 3, Loss: 1.3306
Epoch 4, Loss: 1.2528
Epoch 5, Loss: 1.1968
Epoch 6, Loss: 1.1547
Epoch 7, Loss: 1.1202
Epoch 8, Loss: 1.0932
Epoch 9, Loss: 1.0695
Epoch 10, Loss: 1.0487
MNIST Accuracy: 63.08%
